In [1]:
import numpy as np 
import pandas as pd 
from geopy.geocoders import Nominatim
import time
import requests
import json

In [2]:
data = pd.read_csv("df.csv")

In [35]:
data['station_id'] = data['id'].str.split('/').str[-1]

In [41]:
repeated_station_ids = data['station_id'][data['station_id'].duplicated()]

In [14]:
df = pd.read_csv("df.csv")

In [47]:
data["stationReference"].value_counts()

E7190        3
E73439       2
E6120        2
4063TH       2
E22300       2
E2900        2
4171TH       2
E73639       2
E70139       2
5141TH       1
2063         1
067033       1
E9883        1
44131        1
52101        1
4381TH       1
4127         1
3901         1
43177        1
50151        1
768005       1
0184TH       1
021021       1
E31364       1
E72639       1
51130        1
52210        1
SP00_141B    1
E5850        1
E45851       1
            ..
44120        1
E11480       1
E84189       1
2893TH       1
SU38_73A     1
4189         1
SU74_12      1
SU38_75B     1
49174        1
763201       1
1675TH       1
E22018       1
SU26_14      1
4116         1
AN-SFRB      1
E24925       1
E80535       1
45204        1
E1204        1
2899BTH      1
AN-BURG      1
2490TH       1
694748       1
49117        1
E47061       1
3178         1
E12770       1
E603         1
E15500       1
E23277       1
Name: stationReference, Length: 5381, dtype: int64

In [74]:
data.to_csv("df.csv", index=False)

In [ ]:
data = pd.read_csv("df_UK_lat_lon.csv")

In [115]:
data.columns

Index(['id', 'RLOIid', 'catchmentName', 'dateOpened', 'datumOffset', 'easting',
       'label', 'lat', 'long', 'northing', 'notation', 'riverName',
       'stageScale', 'stationReference', 'status', 'town', 'wiskiID',
       'measures', 'station_id'],
      dtype='object')

In [ ]:
https://environment.data.gov.uk/flood-monitoring/id/stations/1491TH/readings?_sorted&_limit=196

In [ ]:
https://environment.data.gov.uk/flood-monitoring/id/stations/1491TH/measures

In [111]:
data

,id,RLOIid,catchmentName,dateOpened,datumOffset,easting,label,lat,long,northing,notation,riverName,stageScale,stationReference,status,town,wiskiID,measures,station_id
0,http://environment.data.gov.uk/flood-monitorin...,7041,Cotswolds,1994-01-01,NaN,417990,Bourton Dickler,51.874767,-1.740083,219610,1029TH,River Dikler,http://environment.data.gov.uk/flood-monitorin...,1029TH,http://environment.data.gov.uk/flood-monitorin...,Little Rissington,1029TH,"[{""@id"": ""http://environment.data.gov.uk/flood...",1029TH
1,http://environment.data.gov.uk/flood-monitorin...,6022,Welland,1992-01-01,2.00,528000,Surfleet Sluice,52.845991,-0.100848,329300,E2043,River Glen,http://environment.data.gov.uk/flood-monitorin...,E2043,http://environment.data.gov.uk/flood-monitorin...,Surfleet Seas End,L31004,"[{""@id"": ""http://environment.data.gov.uk/flood...",E2043
2,http://environment.data.gov.uk/flood-monitorin...,3072,"Parrett, Brue and West Somerset Streams",1997-01-01,10.00,344383,Gaw Bridge,50.976043,-2.793549,119926,52119,River Parrett,http://environment.data.gov.uk/flood-monitorin...,52119,http://environment.data.gov.uk/flood-monitorin...,Kingsbury Episcopi,520320_FW,"[{""@id"": ""http://environment.data.gov.uk/flood...",52119
3,http://environment.data.gov.uk/flood-monitorin...,6177,Upper and Bedford Ouse,1996-10-01,6.30,529500,Hemingford,52.323618,-0.101287,271200,E21136,River Great Ouse,http://environment.data.gov.uk/flood-monitorin...,E21136,http://environment.data.gov.uk/flood-monitorin...,Hemingford Grey,L33865,"[{""@id"": ""http://environment.data.gov.uk/flood...",E21136
4,http://environment.data.gov.uk/flood-monitorin...,2032,Worcestershire Middle Severn,1976-04-06,NaN,386120,Swindon,52.51274,-2.205945,290560,2067,Smestow Brook,http://environment.data.gov.uk/flood-monitorin...,2067,http://environment.data.gov.uk/flood-monitorin...,Swindon,2067,"[{""@id"": ""http://environment.data.gov.uk/flood...",2067
5,http://environment.data.gov.uk/flood-monitorin...,3158,Fal and St Austell Streams,2000-01-28,13.00,178065,Penryn Trelawney Park,50.172659,-5.109416,34920,48143,Praze Stream,http://environment.data.gov.uk/flood-monitorin...,48143,http://environment.data.gov.uk/flood-monitorin...,Penryn,S19726_FW,"[{""@id"": ""http://environment.data.gov.uk/flood...",48143
6,http://environment.data.gov.uk/flood-monitorin...,5022,Lune and Wyre,1977-01-01,NaN,351210,A6 Bridge,53.858854,-2.743276,440550,720215,River Brock,http://environment.data.gov.uk/flood-monitorin...,720215,http://environment.data.gov.uk/flood-monitorin...,Bilsborrow,720215,"[{""@id"": ""http://environment.data.gov.uk/flood...",720215
7,http://environment.data.gov.uk/flood-monitorin...,8276,Wharfe and Lower Ouse,2001-07-11,NaN,398476,Netherside Hall,54.080705,-2.024779,464978,F1906,River Wharfe,http://environment.data.gov.uk/flood-monitorin...,F1906,http://environment.data.gov.uk/flood-monitorin...,Netherside Hall,F1906,"[{""@id"": ""http://environment.data.gov.uk/flood...",F1906
8,http://environment.data.gov.uk/flood-monitorin...,1152,Ouse,1981-05-01,NaN,533288,Ardingly,51.038451,-0.100268,128283,E8266,River Ouse,http://environment.data.gov.uk/flood-monitorin...,E8266,http://environment.data.gov.uk/flood-monitorin...,Ardingly,352835027,"[{""@id"": ""http://environment.data.gov.uk/flood...",E8266
9,http://environment.data.gov.uk/flood-monitorin...,7080,Vale of White Horse,1991-01-01,NaN,438260,Letcombe Regis,51.571482,-1.449342,185990,1771TH,Letcombe Brook,http://environment.data.gov.uk/flood-monitorin...,1771TH,http://environment.data.gov.uk/flood-monitorin...,Letcombe Regis,1771TH,"[{""@id"": ""http://environment.data.gov.uk/flood...",1771TH


In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5392 entries, 0 to 5391
Data columns (total 19 columns):
id                  5392 non-null object
RLOIid              2526 non-null object
catchmentName       2526 non-null object
dateOpened          2526 non-null object
datumOffset         294 non-null float64
easting             4758 non-null object
label               5392 non-null object
lat                 4758 non-null object
long                4758 non-null object
northing            4758 non-null object
notation            5391 non-null object
riverName           2526 non-null object
stageScale          2526 non-null object
stationReference    5391 non-null object
status              2526 non-null object
town                4665 non-null object
wiskiID             2511 non-null object
measures            5392 non-null object
station_id          5392 non-null object
dtypes: float64(1), object(18)
memory usage: 800.5+ KB


In [107]:
lis = []
for i in data.columns:
    print(data[i][4362])
    lis.append(data[i][4362])

http://environment.data.gov.uk/flood-monitoring/id/stations/
nan
nan
nan
nan
373495
Knightsford Bridge LVL
52.199196
-2.389209
255735
nan
nan
nan
nan
nan
Malvern Hills
nan
[{"@id": "http://environment.data.gov.uk/flood-monitoring/id/measures/-flow--i-15_min-m3_s", "parameter": "flow", "parameterName": "Flow", "period": 900, "qualifier": "", "unitName": "m3/s"}, {"@id": "http://environment.data.gov.uk/flood-monitoring/id/measures/-flow-logged-i-15_min-m3_s", "parameter": "flow", "parameterName": "Flow", "period": 900, "qualifier": "Logged", "unitName": "m3/s"}, {"@id": "http://environment.data.gov.uk/flood-monitoring/id/measures/-level--i-15_min-mAOD", "parameter": "level", "parameterName": "Water Level", "period": 900, "qualifier": "", "unitName": "mAOD"}, {"@id": "http://environment.data.gov.uk/flood-monitoring/id/measures/-level-downstage-i-15_min-m", "parameter": "level", "parameterName": "Water Level", "period": 900, "qualifier": "Downstream Stage", "unitName": "m"}, {"@id": "http:

In [8]:
data.columns

Index(['id', 'RLOIid', 'catchmentName', 'dateOpened', 'datumOffset', 'easting',
       'label', 'lat', 'long', 'northing', 'notation', 'riverName',
       'stageScale', 'stationReference', 'status', 'town', 'wiskiID',
       'measures', 'station_id'],
      dtype='object')

In [7]:
import json
import geopandas as gpd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import ast
from sklearn.cluster import DBSCAN
import numpy as np
import colorsys

app = Dash(__name__)



# Drop rows with missing lat/long
data = data.dropna(subset=['lat', 'long'])

# Function to extract a single float from a list-like string or use the value directly if it's already a float
def extract_float(value):
    if isinstance(value, str):
        try:
            value_list = ast.literal_eval(value)
            return float(value_list[0]) if isinstance(value_list, list) else float(value)
        except (ValueError, SyntaxError):
            return float('nan')
    return float(value)

data['lat'] = data['lat'].apply(extract_float)
data['long'] = data['long'].apply(extract_float)
data = data.dropna(subset=['lat', 'long'])

def generate_colors(n):
    colors = []
    for i in range(n):
        hue = i / n
        rgb = colorsys.hsv_to_rgb(hue, 0.7, 0.9)
        hex_color = '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))
        colors.append(hex_color)
    return colors

def generate_geo_map(plot_data):
    # Load the GeoJSON file
    geojson_file = 'uk.geojson'
    gdf = gpd.read_file(geojson_file)
    
    # Compute center coordinates from the GeoDataFrame
    center_lat = gdf.geometry.centroid.y.mean()
    center_lon = gdf.geometry.centroid.x.mean()
    
    scatter_data = go.Scattermapbox(
        lat=plot_data['lat'],
        lon=plot_data['long'],
        mode='markers',
        marker=dict(
            size=8,
            color=plot_data['point_color'],  # cluster-based color
            opacity=plot_data['opacity'],      # set opacity from the data
            showscale=False
        ),
        text=plot_data.apply(
            lambda row: f"Town: {row['town']}<br>"
                        f"Label: {row['label']}<br>"
                        f"Station ID: {row['station_id']}<br>"
                        f"Notation: {row['notation']}<br>"
                        f"Lat: {row['lat']:.4f}<br>"
                        f"Long: {row['long']:.4f}",
            axis=1
        ),
        hoverinfo='text',
        customdata=plot_data['station_id']
    )
    
    layout = go.Layout(
        margin=dict(l=10, r=10, t=20, b=10, pad=5),
        plot_bgcolor="black",
        paper_bgcolor="black",
        clickmode="event+select",
        hovermode="closest",
        showlegend=False,
        mapbox=go.layout.Mapbox(
            center=go.layout.mapbox.Center(
                lat=center_lat,
                lon=center_lon
            ),
            zoom=4,
            style="carto-positron"
        )
    )
    
    return {"data": [scatter_data], "layout": layout}

@app.callback(
    Output("geo-map", "figure"),
    [Input("geo-map", "clickData")]
)
def update_geo_map(click_data):
    data_modified = data.copy()
    
    # Cluster points using DBSCAN based on lat/lon proximity.
    X = data_modified[['lat', 'long']].to_numpy()
    clustering = DBSCAN(eps=0.01, min_samples=1).fit(X)
    data_modified['cluster'] = clustering.labels_
    
    # Generate colors for each unique cluster so that points in the same cluster share a color.
    unique_clusters = data_modified['cluster'].unique()
    cluster_colors = generate_colors(len(unique_clusters))
    color_mapping = dict(zip(unique_clusters, cluster_colors))
    data_modified['point_color'] = data_modified['cluster'].map(color_mapping)
    
    # Set default opacity to 1 for all points.
    data_modified['opacity'] = 1.0

    # If a point is clicked, decrease opacity of the rest.
    if click_data is not None:
        clicked_station = click_data['points'][0]['customdata']
        # Set lower opacity for all points that are not the clicked one.
        data_modified.loc[data_modified['station_id'] != clicked_station, 'opacity'] = 0.1
        data_modified.loc[data_modified['station_id'] == clicked_station, 'point_color'] = 'black'
    
    return generate_geo_map(data_modified)

app.layout = html.Div([
    dcc.Graph(id='geo-map')
])

if __name__ == '__main__':
    app.run_server(debug=True)


/home/mukullight/anaconda3/envs/cm/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/home/mukullight/anaconda3/envs/cm/lib/python3.6/site-packages/ipykernel_launcher.py:48: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




In [9]:
data.columns 

Index(['id', 'RLOIid', 'catchmentName', 'dateOpened', 'datumOffset', 'easting',
       'label', 'lat', 'long', 'northing', 'notation', 'riverName',
       'stageScale', 'stationReference', 'status', 'town', 'wiskiID',
       'measures', 'station_id'],
      dtype='object')

In [5]:
import pandas as pd

# Create sample data with UK addresses in the "label" field
sample_data = {
    'id': [1, 2, 3, 4],
    'RLOIid': ['RLOI001', 'RLOI002', 'RLOI003', 'RLOI004'],
    'catchmentName': ['Catchment A', 'Catchment B', 'Catchment C', 'Catchment D'],
    'dateOpened': ['2020-01-01', '2021-06-15', '2019-11-20', '2022-03-10'],
    'datumOffset': [0, 0, 0, 0],
    'easting': [123456, 234567, 345678, 456789],
    'label': [
        '10 Downing Street, London, UK',      # London address
        '221B Baker Street, London, UK',       # Famous Sherlock Holmes address
        'Edinburgh Castle, Edinburgh, UK',     # Edinburgh landmark
        'Cardiff Castle, Cardiff, UK'          # Cardiff landmark
    ],
    'lat': [None, None, None, None],  # Missing latitude values to be filled via geocoding
    'long': [None, None, None, None], # Missing longitude values to be filled via geocoding
    'northing': [987654, 876543, 765432, 654321],
    'notation': ['Note1', 'Note2', 'Note3', 'Note4'],
    'riverName': ['River Thames', 'River Fleet', 'River Forth', 'River Taff'],
    'stageScale': ['Scale1', 'Scale2', 'Scale3', 'Scale4'],
    'stationReference': ['Ref1', 'Ref2', 'Ref3', 'Ref4'],
    'status': ['active', 'active', 'inactive', 'active'],
    'town': ['London', 'London', 'Edinburgh', 'Cardiff'],
    'wiskiID': ['WID123', 'WID456', 'WID789', 'WID101'],
    'measures': ['measure1', 'measure2', 'measure3', 'measure4'],
    'station_id': ['ST001', 'ST002', 'ST003', 'ST004']
}

# Create the DataFrame
df_sample = pd.DataFrame(sample_data)

print("Sample UK data generated:")
print(df_sample)


Sample UK data generated:
   id   RLOIid catchmentName  dateOpened  datumOffset  easting  \
0   1  RLOI001   Catchment A  2020-01-01            0   123456   
1   2  RLOI002   Catchment B  2021-06-15            0   234567   
2   3  RLOI003   Catchment C  2019-11-20            0   345678   
3   4  RLOI004   Catchment D  2022-03-10            0   456789   

                             label   lat  long  northing notation  \
0    10 Downing Street, London, UK  None  None    987654    Note1   
1    221B Baker Street, London, UK  None  None    876543    Note2   
2  Edinburgh Castle, Edinburgh, UK  None  None    765432    Note3   
3      Cardiff Castle, Cardiff, UK  None  None    654321    Note4   

      riverName stageScale stationReference    status       town wiskiID  \
0  River Thames     Scale1             Ref1    active     London  WID123   
1   River Fleet     Scale2             Ref2    active     London  WID456   
2   River Forth     Scale3             Ref3  inactive  Edinburgh  WID

In [9]:
import pandas as pd
import googlemaps
import time

# Initialize the Google Maps client
gmaps = googlemaps.Client(key='15502721924490')

def geocode_row(row):
    try:
        # If either lat or long is missing, geocode using the label
        if pd.isnull(row['lat']) or pd.isnull(row['long']):
            geocode_result = gmaps.geocode(row['label'])
            if geocode_result:
                location = geocode_result[0]['geometry']['location']
                row['lat'] = location['lat']
                row['long'] = location['lng']
                
                # Look for county information in the address components
                county = None
                for component in geocode_result[0]['address_components']:
                    if 'administrative_area_level_2' in component['types']:
                        county = component['long_name']
                        break
                row['county'] = county
        else:
            # If lat and long already exist, perform reverse geocoding to obtain county data.
            reverse_result = gmaps.reverse_geocode((row['lat'], row['long']))
            if reverse_result:
                county = None
                for component in reverse_result[0]['address_components']:
                    if 'administrative_area_level_2' in component['types']:
                        county = component['long_name']
                        break
                row['county'] = county
        # Pause briefly to respect API rate limits
        time.sleep(0.1)
    except Exception as e:
        print(f"Error geocoding '{row['label']}': {e}")
        row['county'] = None
    return row


df = df_sample.apply(geocode_row, axis=1)



ValueError: Invalid API key provided.